# Data Challenge - Wiziline
####  by Javier Vallejos Escobar,   jvevallejos@gmail.com

## EDA

In [1]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 59 kB/s 
     |████████████████████████████████| 198 kB 67.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=2f787adf35372fee54db821719ddfd92df36e244bf87bc496dea204161381e1b
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col,row_number
from pyspark.sql.window import Window

spark = SparkSession.builder.appName('spark-data-challenge').getOrCreate()
spark

21/10/03 03:47:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
%%time
import pandas as pd
df = spark.read.csv('../input/profeco-wize/profeco/all_data.csv',header = True,inferSchema=True)
# df = spark.read.csv('../input/profeco-wize/sample.csv',header = True,inferSchema=True)

CPU times: user 173 ms, sys: 40.1 ms, total: 213 ms
Wall time: 2min 31s


In [4]:
# shape of the df
print("Shape : ",(df.count(), len(df.columns)))

Shape :  (62530715, 15)


In [5]:
df.limit(3).toPandas()

,producto,presentacion,marca,categoria,catalogo,precio,fechaRegistro,cadenaComercial,giro,nombreComercial,direccion,estado,municipio,latitud,longitud
0,CUADERNO FORMA ITALIANA,96 HOJAS PASTA DURA. CUADRICULA CHICA,ESTRELLA,MATERIAL ESCOLAR,UTILES ESCOLARES,25.9,2011-05-18 00:00:00.000,ABASTECEDORA LUMEN,PAPELERIAS,ABASTECEDORA LUMEN SUCURSAL VILLA COAPA,CANNES No. 6 ESQ. CANAL DE MIRAMONTES,DISTRITO FEDERAL,TLALPAN,19.29699,-99.125417
1,CRAYONES,CAJA 12 CERAS. JUMBO. C.B. 201423,CRAYOLA,MATERIAL ESCOLAR,UTILES ESCOLARES,27.5,2011-05-18 00:00:00.000,ABASTECEDORA LUMEN,PAPELERIAS,ABASTECEDORA LUMEN SUCURSAL VILLA COAPA,CANNES No. 6 ESQ. CANAL DE MIRAMONTES,DISTRITO FEDERAL,TLALPAN,19.29699,-99.125417
2,CRAYONES,CAJA 12 CERAS. TAMANO REGULAR C.B. 201034,CRAYOLA,MATERIAL ESCOLAR,UTILES ESCOLARES,13.9,2011-05-18 00:00:00.000,ABASTECEDORA LUMEN,PAPELERIAS,ABASTECEDORA LUMEN SUCURSAL VILLA COAPA,CANNES No. 6 ESQ. CANAL DE MIRAMONTES,DISTRITO FEDERAL,TLALPAN,19.29699,-99.125417


In [6]:
df.printSchema()

root
 |-- producto: string (nullable = true)
 |-- presentacion: string (nullable = true)
 |-- marca: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- catalogo: string (nullable = true)
 |-- precio: double (nullable = true)
 |-- fechaRegistro: string (nullable = true)
 |-- cadenaComercial: string (nullable = true)
 |-- giro: string (nullable = true)
 |-- nombreComercial: string (nullable = true)
 |-- direccion: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- latitud: string (nullable = true)
 |-- longitud: string (nullable = true)



## 1. How many commercial chains are monitored, and therefore, included in this database?

In [7]:
df.select("cadenaComercial").distinct().count() 

706

## 2. What are the top 10 monitored products by State?

In [8]:
aux = df.groupBy("estado","producto").count().orderBy("count", ascending=False)
aux.show()

+----------------+--------------------+------+
|          estado|            producto| count|
+----------------+--------------------+------+
|DISTRITO FEDERAL|            REFRESCO|287463|
|DISTRITO FEDERAL|                 FUD|207569|
|          MÉXICO|            REFRESCO|194939|
|DISTRITO FEDERAL|LECHE ULTRAPASTEU...|175640|
|DISTRITO FEDERAL|   DETERGENTE P/ROPA|173452|
|          MÉXICO|                 FUD|149141|
|DISTRITO FEDERAL|             YOGHURT|136720|
|DISTRITO FEDERAL|             CERVEZA|136686|
|          MÉXICO|   DETERGENTE P/ROPA|132862|
|DISTRITO FEDERAL|            MAYONESA|131103|
|DISTRITO FEDERAL|      CHILES EN LATA|130598|
|DISTRITO FEDERAL|    JABON DE TOCADOR|129889|
|DISTRITO FEDERAL|             SHAMPOO|125603|
|DISTRITO FEDERAL|      LECHE EN POLVO|116827|
|          MÉXICO|LECHE ULTRAPASTEU...|116522|
|DISTRITO FEDERAL|         DESODORANTE|113779|
|DISTRITO FEDERAL|               HUEVO|111412|
|DISTRITO FEDERAL|     TOALLA FEMENINA|102356|
|DISTRITO FED

In [9]:
w = Window.partitionBy("estado").orderBy(aux['count'].desc())

top10 = aux.select('*', row_number().over(w).alias('rank')).filter(col('rank') <= 10)

top10.sort(col("estado").desc(), col("count").desc()).show()

# top10.orderBy("estado","count", ascending=False).show()

+---------+--------------------+-----+----+
|   estado|            producto|count|rank|
+---------+--------------------+-----+----+
|   estado|            producto|   20|   1|
|ZACATECAS|   DETERGENTE P/ROPA|20884|   1|
|ZACATECAS|LECHE ULTRAPASTEU...|17309|   2|
|ZACATECAS|           LAVADORAS|16072|   3|
|ZACATECAS|            MAYONESA|15927|   4|
|ZACATECAS|    JABON DE TOCADOR|15926|   5|
|ZACATECAS|                 FUD|15541|   6|
|ZACATECAS|             SHAMPOO|15012|   7|
|ZACATECAS|      CHILES EN LATA|14866|   8|
|ZACATECAS|COMPONENTES DE AUDIO|14799|   9|
|ZACATECAS|            REFRESCO|13925|  10|
|  YUCATÁN|LECHE ULTRAPASTEU...|35991|   1|
|  YUCATÁN|   DETERGENTE P/ROPA|33390|   2|
|  YUCATÁN|            REFRESCO|33235|   3|
|  YUCATÁN|                 FUD|31885|   4|
|  YUCATÁN|           LAVADORAS|27013|   5|
|  YUCATÁN|      CHILES EN LATA|24472|   6|
|  YUCATÁN|            MAYONESA|23629|   7|
|  YUCATÁN|             SHAMPOO|23433|   8|
|  YUCATÁN|            PLANCHAS|

In [10]:
top10.filter(col("estado").contains("DISTRITO FEDERAL")).show()

+----------------+--------------------+------+----+
|          estado|            producto| count|rank|
+----------------+--------------------+------+----+
|DISTRITO FEDERAL|            REFRESCO|287463|   1|
|DISTRITO FEDERAL|                 FUD|207569|   2|
|DISTRITO FEDERAL|LECHE ULTRAPASTEU...|175640|   3|
|DISTRITO FEDERAL|   DETERGENTE P/ROPA|173452|   4|
|DISTRITO FEDERAL|             YOGHURT|136720|   5|
|DISTRITO FEDERAL|             CERVEZA|136686|   6|
|DISTRITO FEDERAL|            MAYONESA|131103|   7|
|DISTRITO FEDERAL|      CHILES EN LATA|130598|   8|
|DISTRITO FEDERAL|    JABON DE TOCADOR|129889|   9|
|DISTRITO FEDERAL|             SHAMPOO|125603|  10|
+----------------+--------------------+------+----+



## 3. Which is the commercial chain with the highest number of monitored products?

In [11]:
df.select("cadenaComercial","producto").distinct().groupBy("cadenaComercial").count()\
                                       .orderBy("count", ascending=False).show()

+--------------------+-----+
|     cadenaComercial|count|
+--------------------+-----+
|             SORIANA| 1059|
|            WAL-MART| 1051|
|MEGA COMERCIAL ME...| 1049|
|  COMERCIAL MEXICANA| 1036|
|            CHEDRAUI| 1026|
|     MERCADO SORIANA| 1024|
|      BODEGA AURRERA| 1012|
|HIPERMERCADO SORIANA| 1006|
|              H.E.B.| 1001|
|        SORIANA PLUS|  999|
|       SORIANA SUPER|  996|
|BODEGA COMERCIAL ...|  979|
|        I.S.S.S.T.E.|  937|
|            SUPERAMA|  936|
|              S MART|  851|
|SUPERMERCADOS SAN...|  849|
|              SUMESA|  848|
|         CITY MARKET|  844|
|FARMACIA GUADALAJARA|  819|
|            CASA LEY|  808|
+--------------------+-----+
only showing top 20 rows



In [12]:
aux = df.groupBy("cadenaComercial","producto").count()

w = Window.partitionBy("cadenaComercial").orderBy(aux['count'].desc())

HighestbyC = aux.select('*', row_number().over(w).alias('rank')).filter(col('rank') <= 10)

# HighestbyC.orderBy("count", ascending=False).show()

Top 10 products of the comertial chain with more products monitored

In [13]:
HighestbyC.filter(col("cadenaComercial").contains("SORIANA")).show()

+--------------------+--------------------+------+----+
|     cadenaComercial|            producto| count|rank|
+--------------------+--------------------+------+----+
|MERCADO EXPRESS S...|            REFRESCO|   709|   1|
|MERCADO EXPRESS S...|   DETERGENTE P/ROPA|   634|   2|
|MERCADO EXPRESS S...|LECHE ULTRAPASTEU...|   526|   3|
|MERCADO EXPRESS S...|     TOALLA FEMENINA|   444|   4|
|MERCADO EXPRESS S...|    JABON DE TOCADOR|   427|   5|
|MERCADO EXPRESS S...|             SHAMPOO|   381|   6|
|MERCADO EXPRESS S...|             CERVEZA|   340|   7|
|MERCADO EXPRESS S...|             YOGHURT|   334|   8|
|MERCADO EXPRESS S...|              ACEITE|   334|   9|
|MERCADO EXPRESS S...|            MAYONESA|   333|  10|
|             SORIANA|            REFRESCO|128758|   1|
|             SORIANA|                 FUD|120610|   2|
|             SORIANA|   DETERGENTE P/ROPA|116610|   3|
|             SORIANA|             SHAMPOO| 94481|   4|
|             SORIANA|    JABON DE TOCADOR| 8938

## 4. Use the data to find an interesting fact.

In [14]:
# which is the product more monitored over all states
p = df.select("estado","producto").distinct().groupBy("producto").count()\
                                       .orderBy("count", ascending=False)
p.show()

+--------------------+-----+
|            producto|count|
+--------------------+-----+
|    TORTILLA DE MAIZ|   35|
|  PAN BLANCO BOLILLO|   35|
|            COLIFLOR|   34|
|                FLAN|   34|
|       GLIBENCLAMIDA|   34|
|            LACTACYD|   34|
|             ESTUFAS|   34|
|         TELMISARTAN|   34|
|           CONTROLIP|   34|
|          AUTOPISTAS|   34|
|             CEBOLLA|   34|
|          PULMONAROM|   34|
|        PRAVASTATINA|   34|
|CAFE TOSTADO Y MO...|   34|
|           BEDOYECTA|   34|
|            AMARYL M|   34|
|HUMULIN  N. INSUL...|   34|
|            PANTOZOL|   34|
|    SAN RAFAEL. REAL|   34|
|            GARBANZA|   34|
+--------------------+-----+
only showing top 20 rows



In [15]:
# which state has more products monitored 
p = df.select("estado","producto").distinct().groupBy("estado").count()\
                                       .orderBy("count", ascending=False)
p.show()

+--------------------+-----+
|              estado|count|
+--------------------+-----+
|              MÉXICO| 1068|
|    DISTRITO FEDERAL| 1063|
|             JALISCO| 1061|
|            TLAXCALA| 1051|
|           QUERÉTARO| 1049|
|             TABASCO| 1046|
|             YUCATÁN| 1046|
|COAHUILA DE ZARAGOZA| 1045|
|              PUEBLA| 1044|
|          GUANAJUATO| 1044|
| MICHOACÁN DE OCAMPO| 1042|
|              SONORA| 1041|
|              OAXACA| 1038|
|             SINALOA| 1038|
|          NUEVO LEÓN| 1038|
|VERACRUZ DE IGNAC...| 1036|
|        QUINTANA ROO| 1033|
|             HIDALGO| 1032|
|           ZACATECAS| 1030|
|              COLIMA| 1016|
+--------------------+-----+
only showing top 20 rows



In [16]:
# missing values by column
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).toPandas()

,producto,presentacion,marca,categoria,catalogo,precio,fechaRegistro,cadenaComercial,giro,nombreComercial,direccion,estado,municipio,latitud,longitud
0,0,0,0,887338,228,20,0,1184,287,0,0,15054,15054,36814,36814


## 5. What are the lessons learned from this exercise?

* Spark it is an excellent tool to work with big data
* Pyspark is easy to use.
* Missing Values  provides wrong values when you do not clean before. 

## 6. Can you identify other ways to approach this problem? Explain.

It is a good dataset and It is possible to make predictions, or evaluate the data set like a time series to find out patterns or anomaly points and also it could  be  applied regression or classification algorithms and clustering.   